## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Havre-Saint-Pierre,CA,2022-08-11 03:09:50,50.2334,-63.5986,57.81,88,0,8.52,clear sky
1,1,Alappuzha,IN,2022-08-11 03:09:51,9.4900,76.3264,82.00,84,96,8.23,overcast clouds
2,2,Tasiilaq,GL,2022-08-11 03:09:52,65.6145,-37.6368,37.56,77,89,3.13,overcast clouds
3,3,Parabel,RU,2022-08-11 03:09:53,58.6978,81.4825,59.85,77,82,8.99,broken clouds
4,4,Santa Cruz,BO,2022-08-11 03:08:28,-17.8000,-63.1667,51.26,76,0,6.91,clear sky


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Alappuzha,IN,2022-08-11 03:09:51,9.4900,76.3264,82.00,84,96,8.23,overcast clouds
6,6,Kununurra,AU,2022-08-11 03:09:54,-15.7667,128.7333,82.38,26,0,17.27,clear sky
11,11,Hithadhoo,MV,2022-08-11 03:09:55,-0.6000,73.0833,82.20,72,3,10.11,clear sky
12,12,Bengkulu,ID,2022-08-11 03:09:56,-3.8004,102.2655,82.54,69,62,2.13,broken clouds
17,17,Cabo San Lucas,MX,2022-08-11 03:09:27,22.8909,-109.9124,85.60,79,98,9.62,overcast clouds
28,28,Kudahuvadhoo,MV,2022-08-11 03:10:01,2.6708,72.8944,81.07,76,7,0.74,clear sky
31,31,Zhuhai,CN,2022-08-11 03:10:03,22.2769,113.5678,82.85,94,40,19.57,shower rain
34,34,Ahuimanu,US,2022-08-11 03:10:05,21.4447,-157.8378,87.51,67,0,17.27,clear sky
37,37,Clifton,US,2022-08-11 03:07:11,40.8584,-74.1638,79.03,74,75,6.91,broken clouds
40,40,San Policarpo,PH,2022-08-11 03:10:08,12.1791,125.5072,84.04,71,13,6.38,few clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Country                213
Date                   213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Alappuzha,IN,82.00,overcast clouds,9.4900,76.3264,
6,Kununurra,AU,82.38,clear sky,-15.7667,128.7333,
11,Hithadhoo,MV,82.20,clear sky,-0.6000,73.0833,
12,Bengkulu,ID,82.54,broken clouds,-3.8004,102.2655,
17,Cabo San Lucas,MX,85.60,overcast clouds,22.8909,-109.9124,
28,Kudahuvadhoo,MV,81.07,clear sky,2.6708,72.8944,
31,Zhuhai,CN,82.85,shower rain,22.2769,113.5678,
34,Ahuimanu,US,87.51,clear sky,21.4447,-157.8378,
37,Clifton,US,79.03,broken clouds,40.8584,-74.1638,
40,San Policarpo,PH,84.04,few clouds,12.1791,125.5072,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": f"{lat},{lng}"}
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Alappuzha,IN,82.00,overcast clouds,9.4900,76.3264,Panoramic Sea Resort
6,Kununurra,AU,82.38,clear sky,-15.7667,128.7333,Hotel Kununurra
11,Hithadhoo,MV,82.20,clear sky,-0.6000,73.0833,Scoop Guest House
12,Bengkulu,ID,82.54,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
17,Cabo San Lucas,MX,85.60,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
28,Kudahuvadhoo,MV,81.07,clear sky,2.6708,72.8944,Niyama Private Islands Maldives
31,Zhuhai,CN,82.85,shower rain,22.2769,113.5678,珠海粤财假日酒店
34,Ahuimanu,US,87.51,clear sky,21.4447,-157.8378,Paradise Bay Resort
37,Clifton,US,79.03,broken clouds,40.8584,-74.1638,Quality Urban Housing
40,San Policarpo,PH,84.04,few clouds,12.1791,125.5072,Agapita M. Nicart


In [18]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
    </dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
#Add the markers layer over the heatmap
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))